<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/getting-started/data-designer-cookbook.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Navigator Data Designer SDK: Getting Started


Welcome! In this guide, we’ll walk through how to use the SDK to generate rich, diverse datasets — from designing your columns to injecting variability and logic into your data.

### 🧙 Magic vs 🛠 Manual Usage

This guide supports two usage styles:

- **Magic Mode** 🪄  
  Enlist the help of an LLM to automatically generate your dataset configuration. Perfect for quick starts and exploring schema ideas.

- **Manual Mode** 🧩  
  Take full control and define your configuration by hand. Ideal when you want precision and complete customization.

---

### 🔁 Generation Methods

Our SDK supports **three flexible generation methods**:

- **Sampling** 🎲  
  Generate values using numerical or categorical distributions to generate realistic, balanced datasets.

- **LLM-generated** 🤖  
  Leverage large language models to create contextual data — such as natural language, code snippets, or structured text.

- **Seeded** 🌱  
  Provide your own seed dataset and sample from it to produce similar or derivative outputs.

---

### 🧠 Expression-Based Columns with Jinja

For more advanced control, you can define **expression-based columns** using Jinja templating. This unlocks:

- 🔀 Conditional logic (`if`, `else`, `for`)
- 🔗 Cross-column references (`{{ column_name }}`)
- ➕ Basic arithmetic and transformations

This lets you dynamically shape values based on other columns and inject logic directly into your data schema.

---

Let’s dive in and start building your dataset! 💡


## Installation and Setup

In [1]:
%%capture
# Install the latest version of Gretel client and dependencies
%pip install -U gretel_client  datasets

In [ ]:
# Import
import time
import pandas as pd

from gretel_client.navigator_client import Gretel
from gretel_client.data_designer import columns as C
from gretel_client.data_designer import params as P


## Create Gretel Client
gretel = Gretel(
    api_key="prompt",
    endpoint="https://api.dev.gretel.ai"
)


model_suite="apache-2.0"
dd = gretel.data_designer.new(model_suite=model_suite) # we will be building on top of this object


# 🌱 Beginner: Kickstart with Magic + Seeded Data

In this section, we’re keeping things simple and powerful. You’ll learn how to get up and running quickly by combining:

- **Magic Mode** 🪄 — Let an LLM generate your dataset configuration for you, based on a few high-level inputs. (Note: Magic is still experimentatal so responses may occasionally be inaccurate)
- **Seeded Generation** 🌾 — Provide a sample dataset, and we’ll draw from it to create new rows with similar structure and variety.

This is a great way to start experimenting with the SDK while getting meaningful output fast. You don’t need to worry about crafting configs by hand just yet — we’ll guide you through using your own data and a touch of AI magic to get impressive results with minimal effort.

📦 By the end of this section, you’ll have a generated dataset built from your seed data and enriched through a config designed by the LLM.

Let’s dive in! 🔍


In [ ]:
# Generated using RNG with add_sampling_column function
dd.magic.add_sampling_column(
    name = "patient",
    description = "An american male living in San Diego",
    interactive=False,
    preview=True
)

# Sampling based off of a distribution
dd.magic.add_sampling_column(
    name = "bmi",
    description = "body mass index of an average person",
    interactive=False,
    preview=True
)

In [ ]:
#Adding Seed Data

# Load the seed dataset

from datasets import load_dataset
df_seed = load_dataset("gretelai/symptom_to_diagnosis")["train"].to_pandas()
df_seed = df_seed.rename(columns={"output_text": "diagnosis", "input_text": "patient_summary"})

print(f"Number of records: {len(df_seed)}")



df_seed.head(n=3)

In [ ]:
# Add seed columns

dd.with_seed_dataset(
        df_seed,
        sampling_strategy="shuffle", #what are the options?
        with_replacement=False,
    )

In [ ]:
dd.preview().dataset.df

In [ ]:
# Add an LLM-generated column. This requires atleast one non-LLM generated column (either through seed or sampling)

dd.magic.add_column(
    name = "api_response",
    instruction_or_description = "An API response that updates a database with with a generated UUID and the diagnosis column",
    must_depend_on=["diagnosis", "patient"],
    interactive=False, #give feedback
    preview=False
)

In [ ]:
# # display sample record
# dd.preview().display_sample_record()

# display sample df
dd.preview().dataset.df

# 🧪 Intermediate: Manual Configuration, Sampling, Jinja & LLM Generation

Now that you’ve gotten a feel for the basics, it’s time to roll up our sleeves and explore more of what the SDK can do.

In this section, we’ll move beyond Magic Mode and **manually create a configuration** to define our dataset structure. Here’s what you’ll be working with:

- **📊 Sampling Columns**  
  Learn how to:
  - Sample values from built-in or custom datasets
  - Use statistical distributions (like normal, uniform, categorical) to shape your data

- **🧠 Expression Columns with Jinja**  
  Add powerful logic and dynamic behavior to your data by:
  - Referencing values from other columns (`{{ other_column }}`)
  - Using conditional logic (`if`, `else`, `for`)
  - Performing basic arithmetic (`+`, `-`, `*`, `/`)

- **🤖 LLM-Based Columns**  
  Inject creativity and contextual intelligence by leveraging an LLM to generate column values like text, summaries, or even simple code.

This section will give you the tools to build smart, expressive datasets that go beyond static values — all by hand. It’s a perfect step toward mastering the full flexibility of the SDK.

Ready to get your hands dirty? 🛠 Let’s go!


#### 👩‍🚀 Person Attributes

| Field | Type | Description |
|-------|------|-------------|
| first_name | str | Person's first name |
| middle_name | str \| None | Person's middle name (optional) |
| last_name | str | Person's last name |
| sex | Sex | Person's sex (enum type) |
| age | int | Person's age |
| zipcode | str | Zipcode/Postal Code |
| street_number | int \| str | Street number (can be numeric or alphanumeric) |
| street_name | str | Name of the street |
| unit | str | Unit/apartment number |
| city | str | City name |
| state | str \| None | State (US locale only) |
| county | str \| None | County (US locale only) |
| country | str | Country name |
| ethnic_background | str \| None | Ethnic background (optional) |
| marital_status | str \| None | Marital status (optional) |
| education_level | str \| None | Education level (optional) |
| bachelors_field | str \| None | Field of bachelor's degree (optional) |
| occupation | str \| None | Occupation (optional) |
| uuid | UUID | Unique identifier |
| locale | str | Locale setting |
| phone_number | PhoneNumber \| None | Generated phone number based on location (None for age < 18) |
| email_address | EmailStr \| None | Generated email address (None for age < 18) |
| birth_date | date | Calculated birth date based on age |
| ssn | str \| None | SSN (US locale only) |

In [ ]:
# Add in sampled columns
dd.add_column(
    C.SamplerColumn(
        name="emergency_contact",
        type=P.SamplingSourceType.PERSON,
        params=P.PersonSamplerParams(locale='en_US',
                                     sex='Female',
                                     city='San Diego')
    )
)


# do vector weights need to sum to 1?
# what happens if sampling category set dont match with subcategory?
# what is scipy distribution?

dd.add_column(
    C.SamplerColumn(
        name="pet_type",
        type=P.SamplingSourceType.CATEGORY,
        params=P.CategorySamplerParams(values=["dog", "cat"],
                                       weights=[0.7, 0.3]),
    )
)

dd.add_column(
    C.SamplerColumn(
        name="first_pet_name",
        type=P.SamplingSourceType.SUBCATEGORY,
        params=P.SubcategoryParams(
            category="pet_type",
            values={
                "dog": ["Buddy", "Max", "Charlie", "Cooper", "Daisy", "Lucy"],
                "cat": ["Oliver", "Leo", "Milo", "Charlie", "Simba", "Luna"],

            }
        )
    )
)

# Sampling from a distribution (e.g. Bernoulli, binomial, poisson, etc..)
dd.add_column(
    C.SamplerColumn(
        name="household_income",
        type=P.SamplingSourceType.POISSON,
        params=P.PoissonSamplerParams(mean=100000)
    )
)

In [ ]:
# Expression
# https://documentation.bloomreach.com/engagement/docs/jinja-syntax

# Referring to existing columns
dd.add_column(
    C.ExpressionColumn(
        name="patient_full_name",
        expr="{{ patient.first_name }} {{ patient.last_name }}"
    )
)

# Deterministically determine outcome based on arithmetic
dd.add_column(
    C.ExpressionColumn(
        name="net_worth",
        expr="{{ household_income - 50000}}"
    )
)


# Conditionally generate the values using expressions based on jinja templating
dd.add_column(
    C.ExpressionColumn(
        name="number_of_children",
        expr="{% if household_income > 100000 %}{{2}}{% else %}1{% endif %}"
    )
)


In [ ]:
dd.preview().dataset.df

In [ ]:
## Add in LLM generated Column

dd.add_column(
    C.LLMGenColumn(
        name="potential_cause",
        type="llm-text",
        prompt=(
            "Write a brief backstory for how {{ patient }} got {{ diagnosis }}."
            "Ensure it is consistent with {{patient_summary}}."
            "Make it no more than 2 sentences."
        ),
    )

)

In [ ]:
dd.preview().dataset.df

# 🧠 Advanced: Bringing It All Together with Complex LLM Prompts

Welcome to the final section — time to go full wizard mode. 🧙‍♂️

Here, we’ll combine everything you've learned so far into a **powerful, LLM-driven workflow**. Instead of manually configuring each column or relying solely on basic sampling, we’ll focus on crafting **rich, detailed prompts** that instruct the LLM to generate entire datasets with structure, logic, and nuance.

In this section, you’ll:

- 🧩 Bring together **Jinja expressions**, and **LLM columns** into a single cohesive config
- ✨ Design and refine complex LLM prompts to guide dataset creation with high fidelity and variability
- 🧠 Leverage the LLM’s contextual understanding to generate multi-column data with realistic relationships and patterns

This approach is ideal when you want to prototype ideas, simulate user behavior, generate synthetic logs, or create diverse, semi-structured content at scale.

By the end of this section, you’ll be able to use the SDK as a creative tool — part data engine, part storytelling assistant.

Let’s take it to the next level. 🚀


In [ ]:
## Add in LLM generated Column

dd.add_column(
    C.LLMGenColumn(
        name="outcome",
        type="llm-text",
        prompt=(
            "Write a brief outcome for what {{patient}} will result in. No more than 1 sentence."
            "{% if bmi > 25 %}"
            "They will have a negative outcome unless they change their lifestyle"
            "They will need the support of {{emergency_contact.first_name}}"
            "{% else %}"
            "They must start spending {{net_worth * 0.5}} dollars on a treatment plan."
            "{% endif %}"
        ),
    )

)

In [ ]:
dd.preview().dataset.df